### Import the libraries

In [180]:
import numpy as np
import pandas as pd
from typing import List

In [2]:
df_students_original = pd.read_excel (r'./student_project_assignment_input-1.xls', sheet_name='studenten')
df_courses_original = pd.read_excel (r'./student_project_assignment_input-1.xls', sheet_name='assignments')


In [3]:
df_students = df_students_original.copy()

In [4]:
df_courses = df_courses_original.copy()

### Clean the data

In [5]:
#stablishing happiness points system

lst_1_8 = list(range(1,9))
lst_1_8.reverse()
lst_1_8
df_students.loc[0][2:] = lst_1_8

In [6]:
#cleaning (students)

df_students.columns = df_students.iloc[0]
df_students = df_students.drop(0)
df_students = df_students.reset_index(drop=True)

In [207]:
clean_empty_rows = df_students['Student ID'].isnull()
df_students_clean = df_students[~clean_empty_rows]

clean_empty_rows2 = df_students_clean['Credits Required'].isnull()
df_students_clean = df_students_clean[~clean_empty_rows2]
df_students_clean.reset_index(drop = True)
df_students_clean = df_students_clean.set_index(['Student ID'])

df_students = df_students_clean

In [8]:
#formatting (students)

credits_num = pd.to_numeric(df_students['Credits Required'])
df_students['Credits Required'] = credits_num

In [9]:
df_students.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 0 to 243
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Student ID        244 non-null    object
 1   Credits Required  244 non-null    int64 
 2   8                 244 non-null    object
 3   7                 244 non-null    object
 4   6                 244 non-null    object
 5   5                 244 non-null    object
 6   4                 244 non-null    object
 7   3                 244 non-null    object
 8   2                 244 non-null    object
 9   1                 244 non-null    object
dtypes: int64(1), object(9)
memory usage: 21.0+ KB


In [10]:
#cleaning (courses)

df_courses.columns = df_courses.iloc[0]
df_courses = df_courses.drop(0)
df_courses = df_courses.reset_index(drop=True)

In [219]:
clean_empty_rows3 = df_courses['Course ID'].isnull()
df_courses_clean = df_courses[~clean_empty_rows3]

df_courses_clean.reset_index(drop = True)
df_courses_clean = df_courses_clean.set_index(['Course ID'])

df_courses = df_courses_clean

In [12]:
#formatting (courses)


df_courses['Min Students'] = pd.to_numeric(df_courses['Min Students'])
df_courses['Max Students'] = pd.to_numeric(df_courses['Max Students'])
df_courses['Time Slot'] = pd.to_numeric(df_courses['Time Slot'])
df_courses['Credits'] = pd.to_numeric(df_courses['Credits'])

In [13]:
df_courses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 0 to 45
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Course ID     46 non-null     object
 1   Min Students  46 non-null     int64 
 2   Max Students  46 non-null     int64 
 3   Time Slot     46 non-null     int64 
 4   Credits       46 non-null     int64 
dtypes: int64(4), object(1)
memory usage: 2.2+ KB


### (Experiment) Order students trying to fill each timeslot first

In [313]:
#make a new dataframe ordered by timeslots

df_courses_timeslots = pd.DataFrame({})

for i in range(df_courses['Time Slot'].max()+1):
    col_name = 'Time Slot ' + str(i)
    df_new_column = pd.DataFrame({col_name : list(df_courses[df_courses['Time Slot'] == i].index)})
    df_courses_timeslots = pd.concat([df_courses_timeslots, df_new_column], axis=1)
    


In [314]:
#THERE MUST BE A MORE GENERAL WAY OF DOING THIS, FIX IN FUTURE.
#ACTUALLY, IGNORE THIS FOR NOW

df_timeslot_0 = df_courses_timeslots[['Time Slot 0']]
df_timeslot_1 = df_courses_timeslots[['Time Slot 1']]
df_timeslot_2 = df_courses_timeslots[['Time Slot 2']]
df_timeslot_3 = df_courses_timeslots[['Time Slot 3']]
df_timeslot_4 = df_courses_timeslots[['Time Slot 4']]
df_timeslot_5 = df_courses_timeslots[['Time Slot 5']]
df_timeslot_6 = df_courses_timeslots[['Time Slot 6']]
df_timeslot_9 = df_courses_timeslots[['Time Slot 9']]

#### Start of doing the artifitial preferences for each course

In [306]:
#Make a dataframe of the credits that the students need, we will update this as we assign them to courses

Credits_remaining = df_students[['Credits Required']].copy()
Credits_remaining

,Credits Required
Student ID,
s_3,3
s_4,3
s_5,3
s_6,3
s_7,3
...,...
s_242,3
s_243,3
s_244,3


In [308]:
def order_students(students: List[str], course:str, current_credits: pd.DataFrame) -> List[str]:
    '''
    Takes in: 
    A list of students that want to sign up for a course
    The course
    The most recent version of credits_remaining
    
    Does:
    Orders a list of students based on these priorities:
    1-the happiness they would get for getting a course 
    2-how many credits they need
    '''
    
    #make a dataframe with the relevant stuff
    #    #select the students
    df_order = df_students.loc[students].copy()
    
    
    #    #make the happiness column
    Happy_list = []
    for i in df_order.index:
        Happy_list += [df_order.loc[i][df_order.loc[i]==course].index[0]]
    
    df_order = df_order.reset_index()
    coll = pd.DataFrame({'Happiness':Happy_list})
    df_order = pd.concat([df_order, coll], axis = 1)
    df_order = df_order.drop(['Credits Required',8,7,6,5,4,3,2,1], axis=1)
    df_order = df_order.set_index(['Student ID'])
    
    
    #    #include current credit necessities
    global Credits_remaining
    credit_col = Credits_remaining.loc[list(df_order.index)]
    df_order = pd.concat([df_order, credit_col], axis = 1)
    
    
    #Order by priorities given
    df_order = df_order.sort_values(['Happiness', 'Credits Required'], ascending=[False, False])
    final_list = list(df_order.index)
    
    return final_list

In [309]:
#Create the students_per_course dataframe

students_per_course = pd.DataFrame({})
for course in df_courses.index:
    stud_list = []
    for stude in df_students.index:
        if course in list(df_students.loc[stude]):
            stud_list += [stude]
    stud_list = order_students(stud_list, course, Credits_remaining)
    col = pd.DataFrame({course:stud_list})
    students_per_course = pd.concat([students_per_course, col], axis = 1)
    

In [187]:
#students_per_course.to_excel('bruteforcing2.xls')

<ipython-input-187-f5941dc2e6a8>:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  students_per_course.to_excel('bruteforcing2.xls')


In [194]:
ajhdvf = [1,2,3,4,5]

In [199]:
print(ajhdvf[1])

two


In [197]:
def funct()->None:
    global ajhdvf
    ajhdvf[1] = 'two'

In [198]:
funct()

### --------IGNORE

In [15]:
#data = {'DA010': ['s_60', 's_132', 's_145', 's_150', 's_153', 's_183', 's_196', 's_243'], 'DA011': ['s_68', 's_90', 's_139', 's_216', 's_223'], 'DA011F': ['s_85', 's_222'], 'DA104': ['s_5', 's_7', 's_8', 's_9', 's_11', 's_12', 's_38', 's_40', 's_42', 's_70', 's_74', 's_84', 's_90', 's_98', 's_102', 's_109', 's_114', 's_125', 's_126', 's_143', 's_151', 's_162', 's_164', 's_165', 's_182', 's_184', 's_191', 's_195', 's_200', 's_202', 's_203', 's_208', 's_211', 's_231', 's_244', 's_246'], 'DA201': ['s_9', 's_10', 's_29', 's_56', 's_58', 's_70', 's_82', 's_87', 's_92', 's_94', 's_109', 's_113', 's_120', 's_128', 's_146', 's_147', 's_156', 's_157', 's_158', 's_167', 's_178', 's_179', 's_185', 's_192', 's_193', 's_197', 's_202', 's_205', 's_207', 's_209', 's_212', 's_240'], 'DA202': ['s_84', 's_137', 's_235', 's_236'], 'DA204': ['s_124', 's_160'], 'DA206': ['s_25', 's_126', 's_128', 's_155', 's_170', 's_246'], 'DA207': ['s_12', 's_76', 's_79', 's_93', 's_104', 's_121', 's_136', 's_138', 's_173', 's_174', 's_177', 's_215', 's_218', 's_220', 's_228', 's_242'], 'DA211': ['s_72', 's_128'], 'DA212': ['s_4', 's_7', 's_11', 's_13', 's_21', 's_38', 's_47', 's_57', 's_59', 's_68', 's_81', 's_87', 's_95', 's_98', 's_100', 's_110', 's_124', 's_126', 's_135', 's_142', 's_144', 's_145', 's_148', 's_174', 's_176', 's_180', 's_185', 's_188', 's_204', 's_209', 's_216', 's_227', 's_230', 's_237'], 'DA217': ['s_24', 's_43', 's_51', 's_57', 's_61', 's_152', 's_153', 's_177', 's_180', 's_182', 's_201', 's_203', 's_230', 's_237', 's_241'], 'DA220': [], 'DA234': ['s_27', 's_28', 's_184', 's_198', 's_222', 's_223', 's_225'], 'DA238': ['s_105', 's_184'], 'DA250F-1': ['s_14', 's_181'], 'DA250F-2': ['s_173', 's_176'], 'DA251F': ['s_24', 's_47', 's_183', 's_236'], 'DA301-1': ['s_6', 's_8', 's_33', 's_34', 's_64', 's_66', 's_83', 's_129', 's_130', 's_146', 's_155', 's_161', 's_162', 's_163', 's_178', 's_190', 's_195', 's_201', 's_237'], 'DA302': ['s_3', 's_18', 's_36', 's_44', 's_89', 's_151', 's_186', 's_195', 's_227', 's_243'], 'DA306': ['s_27', 's_28', 's_112', 's_150', 's_157', 's_168', 's_176', 's_187', 's_204'], 'DA308': ['s_26', 's_86', 's_133', 's_158', 's_213'], 'DA311': ['s_11', 's_17', 's_32', 's_34', 's_40', 's_41', 's_55', 's_77', 's_83', 's_89', 's_107', 's_123', 's_127', 's_141', 's_150', 's_154', 's_159', 's_168', 's_170', 's_193', 's_198', 's_206', 's_224', 's_229', 's_239', 's_244'], 'DA403': ['s_33', 's_48', 's_102', 's_108', 's_112', 's_135', 's_172', 's_239'], 'DA408': ['s_55', 's_60', 's_160', 's_216', 's_234', 's_242', 's_245'], 'DA415F': ['s_14', 's_19', 's_20', 's_22', 's_36', 's_41', 's_54', 's_57', 's_61', 's_65', 's_72', 's_86', 's_96', 's_97', 's_99', 's_102', 's_108', 's_111', 's_114', 's_118', 's_121', 's_137', 's_149', 's_162', 's_166', 's_169', 's_192', 's_198', 's_201', 's_215', 's_221', 's_222', 's_231', 's_232', 's_240'], 'DA511': ['s_16', 's_22', 's_23', 's_42', 's_45', 's_89', 's_90', 's_95', 's_108', 's_124', 's_127', 's_139', 's_155', 's_161', 's_163', 's_164', 's_175', 's_187', 's_196', 's_202', 's_230', 's_234'], 'DA512': ['s_3', 's_4', 's_8', 's_14', 's_15', 's_19', 's_20', 's_25', 's_30', 's_48', 's_66', 's_80', 's_82', 's_88', 's_89', 's_99', 's_101', 's_104', 's_118', 's_120', 's_122', 's_127', 's_129', 's_137', 's_159', 's_161', 's_167', 's_172', 's_217', 's_220', 's_235', 's_236', 's_238'], 'DA513': ['s_16', 's_45', 's_96', 's_100', 's_106', 's_142', 's_148', 's_149', 's_166', 's_196', 's_240'], 'DA605': ['s_9', 's_12', 's_23', 's_29', 's_30', 's_33', 's_41', 's_45', 's_51', 's_68', 's_74', 's_88', 's_91', 's_100', 's_114', 's_125', 's_151', 's_152', 's_166', 's_204', 's_211', 's_214', 's_232', 's_238', 's_244'], 'DA701': ['s_17', 's_26', 's_44', 's_54', 's_58', 's_67', 's_73', 's_81', 's_93', 's_94', 's_103', 's_116', 's_119', 's_136', 's_181', 's_186', 's_194', 's_199', 's_208', 's_228', 's_241'], 'DA702': ['s_81', 's_88', 's_116', 's_120', 's_130', 's_152', 's_171', 's_181', 's_226'], 'DA704': ['s_27', 's_28', 's_35', 's_37', 's_49', 's_53', 's_67', 's_69', 's_73', 's_78', 's_111', 's_140', 's_171', 's_182', 's_189', 's_199', 's_200', 's_206', 's_210', 's_233'], 'DA707': ['s_3', 's_14', 's_18', 's_23', 's_31', 's_35', 's_36', 's_37', 's_40', 's_44', 's_49', 's_52', 's_53', 's_59', 's_62', 's_63', 's_65', 's_66', 's_73', 's_77', 's_78', 's_92', 's_93', 's_94', 's_97', 's_99', 's_101', 's_103', 's_104', 's_107', 's_111', 's_112', 's_116', 's_117', 's_122', 's_130', 's_134', 's_136', 's_140', 's_141', 's_147', 's_149', 's_156', 's_159', 's_163', 's_168', 's_170', 's_179', 's_186', 's_194', 's_210', 's_212', 's_218', 's_226', 's_233', 's_238', 's_241'], 'DA714': ['s_10', 's_20', 's_22', 's_24', 's_31', 's_32', 's_34', 's_40', 's_48', 's_50', 's_55', 's_59', 's_61', 's_62', 's_72', 's_91', 's_97', 's_103', 's_105', 's_117', 's_132', 's_154', 's_180', 's_210', 's_231'], 'DA717': ['s_15', 's_16', 's_43', 's_56', 's_79', 's_91', 's_106', 's_117', 's_132', 's_138', 's_139', 's_143', 's_148', 's_160', 's_169', 's_188', 's_191', 's_192', 's_205', 's_222', 's_224', 's_227', 's_229', 's_231', 's_234', 's_245'], 'DA718': ['s_24', 's_46', 's_50', 's_101', 's_110', 's_118', 's_119', 's_135', 's_165', 's_177', 's_191', 's_215', 's_219'], 'DA719-1': ['s_21', 's_39', 's_51', 's_80', 's_115', 's_154', 's_169', 's_175'], 'DA719-2': ['s_115'], 'DA803': ['s_5', 's_6', 's_13', 's_38', 's_54', 's_71', 's_123', 's_144', 's_153', 's_190', 's_219'], 'DA806': ['s_19', 's_46', 's_64', 's_85', 's_108', 's_115', 's_123', 's_157', 's_164', 's_165', 's_211', 's_221', 's_243'], 'DA910-1': ['s_15', 's_17', 's_46', 's_47', 's_50', 's_52', 's_63', 's_77', 's_106', 's_107', 's_113', 's_119', 's_121', 's_134', 's_147', 's_172', 's_183', 's_194', 's_203', 's_214', 's_235', 's_239'], 'DA910-2': ['s_7', 's_18', 's_31', 's_32', 's_42', 's_52', 's_53', 's_62', 's_65', 's_69', 's_76', 's_80', 's_83', 's_84', 's_122', 's_134', 's_141', 's_142', 's_143', 's_156', 's_208', 's_213', 's_214', 's_219', 's_221', 's_226', 's_233'], 'DA912': ['s_4', 's_6', 's_21', 's_25', 's_45', 's_60', 's_63', 's_64', 's_74', 's_75', 's_85', 's_95', 's_96', 's_98', 's_105', 's_125', 's_131', 's_144', 's_145', 's_175', 's_188', 's_190', 's_223', 's_224', 's_225', 's_229', 's_246'], 'DI702': [], 'DI703': ['s_30', 's_66', 's_71', 's_95', 's_129', 's_189', 's_197', 's_207']}

In [16]:

#key_list = list(data)

In [17]:
'''
df_solution = pd.DataFrame({})

for i in range(len(list(data))):
    new_col = pd.DataFrame({str(key_list[i]) : data[list(data)[i]]  }) 
    df_solution = pd.concat([df_solution, new_col], axis = 1)
    
df_solution'''

'\ndf_solution = pd.DataFrame({})\n\nfor i in range(len(list(data))):\n    new_col = pd.DataFrame({str(key_list[i]) : data[list(data)[i]]  }) \n    df_solution = pd.concat([df_solution, new_col], axis = 1)\n    \ndf_solution'

In [18]:
 #pd.set_option("display.max_rows", None, "display.max_columns", None)

In [20]:
#df_solution

### Keep track of changes/DataFrames

In [208]:
df_students

,Credits Required,8,7,6,5,4,3,2,1
Student ID,,,,,,,,,
s_3,3,DA707,DA512,DA302,DA702,DA714,DA415F,DA910-2,DA311
s_4,3,DA212,DA912,DA512,DA605,DA714,DA302,DA104,DA311
s_5,3,DA104,DA803,DA217,DA306,DA311,DA212,DA806,DA513
s_6,3,DA912,DA301-1,DA803,DA717,DA104,DA513,DA217,DA605
s_7,3,DA910-2,DA212,DA104,DA302,DA311,DA206,DA251F,DA803
...,...,...,...,...,...,...,...,...,...
s_242,3,DA207,DA408,DA912,DA714,DA803,DA717,DA719-1,DA415F
s_243,3,DA806,DA010,DA302,DA605,DA217,DA204,DA251F,DA415F
s_244,3,DA104,DA311,DA605,DA717,DA201,DA511,DA302,DA206


In [220]:
df_courses

,Min Students,Max Students,Time Slot,Credits
Course ID,,,,
DA010,5,20,2,1
DA011,5,20,0,1
DA011F,10,24,3,1
DA104,5,20,9,1
DA201,5,20,1,2
DA202,5,20,5,1
DA204,4,60,9,1
DA206,12,20,1,1
DA207,5,20,3,2


In [311]:
df_courses_timeslots

,Time Slot 0,Time Slot 1,Time Slot 2,Time Slot 3,Time Slot 4,Time Slot 5,Time Slot 6,Time Slot 7,Time Slot 8,Time Slot 9
0,DA011,DA201,DA010,DA011F,DA707,DA202,DA212,NaN,NaN,DA104
1,NaN,DA206,DA217,DA207,DA803,DA211,DA302,NaN,NaN,DA204
2,NaN,DA250F-1,DA250F-2,DA220,NaN,DA238,DA308,NaN,NaN,DA311
3,NaN,DA512,DA704,DA234,NaN,DA301-1,DA408,NaN,NaN,DA403
4,NaN,DA718,DA910-2,DA251F,NaN,DA306,DA701,NaN,NaN,DA415F
5,NaN,DA912,NaN,DA511,NaN,DA605,DA719-2,NaN,NaN,DA513
6,NaN,NaN,NaN,DA714,NaN,DA717,DA806,NaN,NaN,DA702
7,NaN,NaN,NaN,DA910-1,NaN,DA719-1,NaN,NaN,NaN,DI703
8,NaN,NaN,NaN,NaN,NaN,DI702,NaN,NaN,NaN,NaN


In [312]:
students_per_course

,DA010,DA011,DA011F,DA104,DA201,DA202,DA204,DA206,DA207,DA211,...,DA718,DA719-1,DA719-2,DA803,DA806,DA910-1,DA910-2,DA912,DI702,DI703
0,s_153,s_223,s_222,s_5,s_157,s_84,s_160,s_128,s_76,s_72,...,s_46,s_39,s_115,s_71,s_19,s_15,s_7,s_6,s_118,s_66
1,s_60,s_90,s_29,s_8,s_10,s_236,s_124,s_126,s_138,s_128,...,s_165,s_21,s_74,s_5,s_85,s_50,s_18,s_21,s_193,s_129
2,s_132,s_139,s_85,s_9,s_82,s_137,s_20,s_25,s_173,s_70,...,s_24,s_115,s_229,s_13,s_115,s_52,s_32,s_60,s_50,s_189
3,s_243,s_68,s_84,s_12,s_87,s_235,s_151,s_155,s_220,s_109,...,s_50,s_154,s_26,s_6,s_243,s_107,s_42,s_64,s_77,s_197
4,s_86,s_78,s_172,s_38,s_109,s_76,s_162,s_170,s_228,s_196,...,s_101,s_175,s_175,s_38,s_64,s_113,s_65,s_96,s_80,s_207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,NaN,NaN,NaN,s_204,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,NaN,NaN,NaN,s_205,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,NaN,NaN,NaN,s_228,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,NaN,NaN,NaN,s_229,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
